## 03_Clasificación del conjunto MNIST

Vamos a completar los tutoriales anteriores con el entrenamiento de una red neuronal para tratar de resolver un problema de **clasificación multiclase**.

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist

### Obtenemos y visualizamos el dataset

Obtenemos el dataset del conjunto de imágenes MINIST desde Keras.

Se trata de un conjunto de imágenes de números escritos a mano del 0 al 9 dividido en dos subconjuntos:
- Subconjunto de entrenamiento: formado por 60.000 imágenes de 28x28 píxeles.
- Subconjunto de test: formado por 10.000 imágenes de 28x28 píxeles.

Todas las imágenes son en blanco y negro, con valores en cada pixel entre 0 y 255.

Hay 10 clases en las imágenes correspondientes a los números del 0 al 9.

Las distribución de las clases está equilibrada en los subconjuntos de entrenamiento, con unas 6000 muestras por clase, y de test, con unas 1000 muestras por clase.

In [ ]:
# descargamos el dataset MNIST
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
# obtenemos las dimensiones de las imágenes del subconjunto de entrenamiento
print(X_train.shape)

In [ ]:
# obtenemos las dimensiones de las etiquetas del subconjunto de entrenamiento
print(y_train.shape)

In [ ]:
# obtenemos las dimensiones de las imágenes del subconjunto de test
print(X_test.shape)

In [ ]:
# obtenemos las dimensiones de las etiquetas del subconjunto de test
print(y_test.shape)

In [ ]:
# dimensiones de una imagen individual
X_train[0].shape

In [ ]:
print('Valor máximo del pixel:', np.max(X_train[0]))
print('Valor mínimo del pixel:', np.min(X_train[0]))

In [ ]:
# obtenemos los valores únicos del numpy array
print(np.unique(y_train))

In [ ]:
# creamos una figura con 2 filas y 5 columnas de imágenes
fig, axes = plt.subplots(3, 5, figsize=(12, 5))
axes = axes.ravel()

# iteramos sobre 15 imágenes
for i in np.arange(0, 15):
    axes[i].imshow(X_train[i], cmap='gray')
    axes[i].set_title(f"Label: {y_train[i]}", fontsize=12)
    plt.subplots_adjust(hspace=0.5)
    axes[i].axis('off')

plt.show()

In [ ]:
# vemos la distribución de imágenes para cada número en el conjunto de datos de entrenamiento
plt.figure(figsize=(10, 5))
plt.hist(y_train, bins=10, rwidth=0.8, color='blue', alpha=0.7)
plt.title('Distribución de etiquetas en el conjunto de entrenamiento MNIST')
plt.xlabel('Etiquetas')
plt.ylabel('Cantidad')
plt.xticks(np.arange(0, 10))
plt.grid(axis='y')
plt.show()

In [ ]:
# vemos la distribución de imágenes para cada número en el conjunto de datos de test
plt.figure(figsize=(10, 5))
plt.hist(y_test, bins=10, rwidth=0.8, color='blue', alpha=0.7)
plt.title('Distribución de etiquetas en el conjunto de test MNIST')
plt.xlabel('Etiquetas')
plt.ylabel('Cantidad')
plt.xticks(np.arange(0, 10))
plt.grid(axis='y')
plt.show()

### Transformamos los datos

Vamos a realizar las siguientes transformaciones en las imágenes, previamente a pasarlas por la red neuronal:
- Pasamos las imágenes de matrices de 28x28 a vectores de longitud 784
- Convertimos los numeros enteros de los píxeles a numeros reales
- Normalizamos las imágenes, de manera que los valores de cada pixel pasa de estar entre 0 y 255 a estar entre 0 y 1. Para ello, dividimos los valores de la intensidad de cada pixel entre 255. Esta transnformación le va a dar estabilidad al entrenamiento de la red.

Además, vamos a comprobar las dimensiones resultantes de los datasets.

In [ ]:
# modificamos las imágenes del subconjunto de entrenamiento
X_train_model = X_train.reshape(60000,784).astype('float32')/255.0

# modificamos las imágenes del subconjunto de test
X_test_model = X_test.reshape(10000,784).astype('float32')/255.0

print('Dimensiones de las imágenes del subconjunto de entrenamiento: ', X_train_model.shape)
print('Dimensiones de las imágenes del subconjunto de test: ', X_test_model.shape)

### Definimos la red neuronal

Como red neuronal vamos a definir una red con una entrada de un vector de 784 dimensiones en una capa densa formada por 100 neuronas con activación ReLU; tras esto le aplicamos una regularización dropout del 30% y lo pasamos a una capa densa de 50 neuronas con activación ReLU, seguida de una capa de Dropout del 30%. Finalmente tenemos una capa de salida con 10 neuronas y una activación softmax que transforma los valores de salida en probabilidades para la clasificación multiclase.

Usaremos para la definición del modelo el modo Secuencial.

In [ ]:
# definimos el modelo de red neuronal
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(100, activation='relu', input_shape=(784,)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
# definimos el optimizador Adam
optimizer_model = tf.keras.optimizers.Adam(learning_rate=0.0001)
# definimos la función de pérdida como Spase Categorical Crossentropy
# ya que las etiquetas de las clases nos vienen dadas como números enteros
loss_model = tf.keras.losses.SparseCategoricalCrossentropy()

# compilamos el modelo, usando el optimizador y la pérdida definidas y como métrica la accuracy
model.compile(optimizer=optimizer_model, loss=loss_model, metrics=['accuracy'])

In [ ]:
# entrenamos el modelo con los datos de entrenamiento y de validacion
history = model.fit(x=X_train_model,
                    y=y_train,
                    batch_size=32,
                    epochs=50,
                    validation_data=(X_test_model, y_test))

In [ ]:
# obtenemos la información del entrenamiento y la validación
training_accuracy = history.history['accuracy']
training_loss = history.history['loss']
validation_accuracy = history.history['val_accuracy']
validation_loss = history.history['val_loss']
epochs = range(1, len(training_accuracy) + 1)

In [ ]:
# dibujamos la evolución de la accuracy
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, training_accuracy, label='Training accuracy', color='blue')
plt.plot(epochs, validation_accuracy, label='Validation accuracy', color='green')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# dibujamos la evolución de la pérdida
plt.subplot(1, 2, 2)
plt.plot(epochs, training_loss, label='Training loss', color='blue')
plt.plot(epochs, validation_loss, label='Validation loss', color='green')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

### Inferencia

Con el modelo entrenado, vamos a hacer inferencia sobre algunas imágenes del subconjunto de test y visualizar si la imagen y la etiqueta se corresponde con la predicción.

In [ ]:
# cogemos las primeras 10 imágenes del subconjunto de test
sample_images = X_test_model[:10]

In [ ]:
# realizamos las predicciones con el modelo entrenado
predicted_probs = model.predict(sample_images)
predicted_labels = np.argmax(predicted_probs, axis=1)

In [ ]:
# creamos una figura con 2 filas y 5 columnas
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
axes = axes.ravel()

# iteramos sobre las 10 imágenes
for i in np.arange(0, 10):
    axes[i].imshow(X_test[i], cmap='gray')
    axes[i].set_title(f"Predicción: {predicted_labels[i]}\nVerdadero: {y_test[i]}", fontsize=12)
    plt.subplots_adjust(hspace=1)
    axes[i].axis('off')

plt.show()